# Azure Storage Service Demo
This is intended to be run on the Python 2 kernel.   
If you have not installed azure and azure-storage un-comment the pip below and run it.

In [1]:
!pip install --upgrade azure-storage==0.32.0

    100% |████████████████████████████████| 163kB 3.5MB/s eta 0:00:01
    100% |████████████████████████████████| 256kB 2.7MB/s eta 0:00:01
    100% |████████████████████████████████| 71kB 7.4MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 4.4MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 6.0MB/s eta 0:00:01
    100% |████████████████████████████████| 153kB 4.0MB/s eta 0:00:01
    100% |████████████████████████████████| 184kB 3.5MB/s eta 0:00:01
  Found existing installation: six 1.10.0
    Uninstalling six-1.10.0:
      Successfully uninstalled six-1.10.0
  Found existing installation: python-dateutil 2.6.0
    Uninstalling python-dateutil-2.6.0:
      Successfully uninstalled python-dateutil-2.6.0
  Found existing installation: certifi 2017.1.23
    DEPRECATION: Uninstalling a distutils installed project (certifi) has been deprecated and will be removed in a future version. This is due to the fact that uninstalling a distutils project will only p

In [2]:
import csv
import sys
import azure.storage
from azure.storage.table import TableService, Entity
from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess
import time
print("hello at" + time.asctime(time.localtime(time.time())))

hello atWed Feb 21 08:36:45 2024


follow the instructions in the book to create a storage account.   the one used here is "tutorial", but you will want to pick something else and replace that name everywhere below

In [7]:
key = 'ends with ==' #replace with key
account = '' #replace with account name
block_blob_service = BlockBlobService(account_name=account,
    account_key=key)
block_blob_service.create_container('datacont', 
                                    public_access=PublicAccess.Container)

False

In [10]:
table_service = TableService(account_name=account, 
                             account_key=key)

In [11]:
if table_service.create_table('DataTable'):
    print("table created")
else:
    print("table already there")

table already there


Now you need a directory with a csv file.  For example

'experiment1', '1', '3/15/2002', 'exp1', 'this is the comment'

'experiment1', '2', '3/15/2002', 'exp2', 'this is the comment2'

'experiment2', '3', '3/16/2002', 'exp3', 'this is the comment3'

'experiment3', '4', '3/16/2002', 'exp4', 'this is the comment233'

now you need a directory datafiles that has four blobs of any type.  call them exp1, exp2, exp3, exp4.  
we have one ready for you if you are using the tutorial container. It is stored in /datadir.
If you are running this somewhere else you will need to make your own.  
let's see what is there.

In [12]:
 with open('/datadir/experiments.csv', 'rb') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        print(item)

['experiment1', ' id1', ' 12/1/2016', 'a.jpg', ' "here is a view of what is to be made"']
['experiment1', ' id2', ' 12/3/2016', 'b.jpg', ' "map rededuce picture"']
['experiment2', ' id3', ' 12/4/2016', 'c.jpg', ' "sample notebook"']
['experiment3', ' id4', ' 12/5/2016', 'd.jpg', ' "workers"']
['experiment1', ' id5', ' 12/6/2016', 'e.jpg', ' "bio samples"']


the code below will upload the data blobs into the container "datacont", then create the url and upload the metadata into the table "DataTable".

In [15]:
with open('/datadir/experiments.csv', 'rb') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for item in csvf:
        print(item)
        block_blob_service.create_blob_from_path(
            'datacont', item[3], "/datadir/"+item[3]
        )
        url = "https://"+account+".blob.core.windows.net/datacont/"+item[3]
        metadata_item = {
            'PartitionKey': item[0], 'RowKey': item[1],
            'description' : item[4], 'date' : item[2], 'url':url
        }
        #insert_or_replace_entity instead of insert_entity
        table_service.insert_or_replace_entity('DataTable', metadata_item)


['experiment1', ' id1', ' 12/1/2016', 'a.jpg', ' "here is a view of what is to be made"']
['experiment1', ' id2', ' 12/3/2016', 'b.jpg', ' "map rededuce picture"']
['experiment2', ' id3', ' 12/4/2016', 'c.jpg', ' "sample notebook"']
['experiment3', ' id4', ' 12/5/2016', 'd.jpg', ' "workers"']
['experiment1', ' id5', ' 12/6/2016', 'e.jpg', ' "bio samples"']


Next let's do a querry for experiment1 and project onto the urls.  it should print the urls created above

In [16]:
tasks = table_service.query_entities('DataTable', filter="PartitionKey eq 'experiment1'", select='url')
for task in tasks:
    print(task.url)

https://annacc.blob.core.windows.net/datacont/a.jpg
https://annacc.blob.core.windows.net/datacont/b.jpg
https://annacc.blob.core.windows.net/datacont/e.jpg


The output should look somethink like

https://escistore.blob.core.windows.net/datacont/exp1

https://escistore.blob.core.windows.net/datacont/exp2

try clicking on the links in your output


Next download the "azure storage explorer" and look at you table.

You will need to add the account with the key to the storage explore. 